# Compare the DBNNs to Other Methods on Real-World Samples from Sets $\mathcal{R}_{height}$ and $\mathcal{R}_{earthquake}$

## Set up the Environment

In [1]:
# Import everything that's needed to run the notebook
import os
import pickle
import dill
import pathlib
import datetime
import random
import time

from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import util
import robust
from ipynb.fs.defs.descriptor_based_neural_networks import DescriptorBuilder
from ipynb.fs.defs.construct_sbnn import SBNNPreprocessor
from sklearn.model_selection import learning_curve

plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.rc('axes', labelsize=15)

In [2]:
# Define the path to the configuration dictionary
config_path = 'configuration.p'

# Load the configuration dictionary
with open(config_path, 'rb') as f:
    configuration = pickle.load(f)
    
# Get the paths to the relevant directories 
data_directory_path = configuration['data']['directory_path']
classifiers_directory_path = configuration['classifiers']['directory_path']

## Load the Storage of Results and Reports

In [3]:
dbnn_storage = {}

results_directory_path = configuration['results']['directory_path']
path = os.path.join(results_directory_path, 'dbnn_results.p')
with open(path, 'rb') as f:
    dbnn_storage['results'] = pickle.load(f)

reports_directory_path = configuration['reports']['directory_path']
path = os.path.join(reports_directory_path, 'dbnn')
path = os.path.join(path, 'dbnn_reports.p')
with open(path, 'rb') as f:
    dbnn_storage['reports'] = pickle.load(f)

## Load the DBNNs

In [4]:
with open('dbnns1.p', 'rb') as f:
    dbnns = dill.load(f)

## Load and Prepare the Data

In [5]:
# Define the dictionary to store the actual datasets, indexed by their names
datasets = {}

# Load the datasets
for set_name in ['R_height', 'R_earthquake']:
    set_path = configuration['data']['datasets'][set_name]['path']
    print('Loading {} from {}'.format(set_name, set_path))
    datasets[set_name] = util.load_from_file(set_path)
    print('Done.')

Loading R_height from data/R_height.data
Done.
Loading R_earthquake from data/R_earthquake.data
Done.


In [6]:
for set_name in datasets:
    labels = [sample.pop() for sample in datasets[set_name]]
    samples = datasets[set_name]
    
    datasets[set_name] = {'samples' : samples, 'labels' : labels}

## Load the Tests

In [7]:
# Make a dictionary to hold the tests
test_classifiers = {}

# Specify the classical tests
codes = ['SW', 'SF', 'LF', 'JB', 'DP', 'AD', 'CVM', 'FSSD']

# Load the classical tests
for test_code in codes:
    test, statistic = util.get_test(test_code)
    for alpha in [0.01, 0.05]:
        test_classifiers[(test_code, alpha)] = util.TestClassifier(test, statistic, alpha)

# Although SBNN is not technically a test, consider it too.
with open(os.path.join('classifiers', 'sbnn.p'), 'rb') as f:
    sbnn = pickle.load(f)

test_classifiers[('SBNN', '/')] = sbnn
codes += ['SBNN']

# Specify the robustified tests
robust_codes = ['MMRT1', 'MMRT2', 'TTRT1', 'TTRT2', 'RSW', 'RLM']

# Load the robustified tests
for test_code in robust_codes:
    test, statistic = robust.get_robust_test(test_code)
    for alpha in [0.01, 0.05]:
        test_classifiers[(test_code, alpha)] = util.TestClassifier(test, statistic, alpha)

## Evaluate the Tests

### $\mathcal{R}_{height}$

In [8]:
samples_lt10 = [sample for sample in samples if len(sample) < 10]
m_lt10 = len(samples_lt10)
samples_ge10 = [sample for sample in samples if len(sample) >= 10]
m_ge10 = len(samples_ge10)

In [28]:
results = {}

for code in test_classifiers:
    print(code)
    classifier = test_classifiers[code]
    
    start = time.time()
    df_lt10 = util.evaluate_pretty(samples_lt10, [1 for l in range(m_lt10)], 
                                   classifier, 
                                   metrics=['TPR', 'UR'])
    end = time.time()
    display(df_lt10.loc[0, 'UR'])
    
    start = time.time()
    df_ge10 = util.evaluate_pretty(samples_ge10, [1 for l in range(m_ge10)], 
                                   classifier, 
                                   metrics=['TPR', 'UR'])
    end = time.time()
    display(df_ge10.loc[0, 'UR'])
    
    results[code] = {'n < 10' : df_lt10.loc[0, 'TPR'], 'n >= 10' : df_ge10.loc[0, 'TPR']}

('SW', 0.01)


0.0

0.0

('SW', 0.05)


0.0

...

0.0

0.0

('RLM', 0.05)


0.0

0.0

In [51]:
df = pd.DataFrame(results).T
df.columns = ['$n < 10$', '$n \geq 10$']

In [52]:
df_res = pd.concat([dbnn_storage['results']['evaluation']['R_height']['df'], df])
df_res

$n < 10$  $n \geq 10$
DBNN  0.01  1.000000     1.000000
      0.05  1.000000     1.000000
      0.1   0.928571     0.979167
      /     0.357143     0.708333
SW    0.01  0.933000     0.118895
      0.05  0.824000     0.062105
SF    0.01  0.941000     0.135316
      0.05  0.813000     0.072632
LF    0.01  0.946000     0.243421
      0.05  0.867000     0.137684
JB    0.01  1.000000     0.255632
      0.05  1.000000     0.181421
DP    0.01       NaN     0.230053
      0.05       NaN     0.144632
AD    0.01  1.000000     0.133316
      0.05  0.988000     0.072368
CVM   0.01  0.000000     0.163895
      0.05  0.000000     0.092789
FSSD  0.01  0.983000     0.774368
      0.05  0.970000     0.608105
SBNN  /     0.285000     0.051842
MMRT1 0.01  0.968000     0.228316
      0.05  0.857000     0.100368
MMRT2 0.01  0.979000     0.271947
      0.05  0.900000     0.104158
TTRT1 0.01  0.944000     0.639421
      0.05  0.841000     0.475474
TTRT2 0.01  0.945000     0.228684
      0.05  0.838000     0.128895
RSW   0.01  0.948000     0.503211
      0.05  0.859000     0.362263
RLM   0.01  0.998000     0.996316
      0.05  0.992000     0.994526

In [57]:
print(util.get_latex_table(df_res, index=True, caption='$\mathcal{R}_{height}$', label='tab:height'))

\begin{table}
\centering
\caption{$\mathcal{R}_{height}$}
\label{tab:height}
\begin{tabular}{llrr}
\toprule
    &      &  $n < 10$ &  $n \geq 10$ \\
\midrule
DBNN & 0.01 &   $1.000$ &      $1.000$ \\
    & 0.05 &   $1.000$ &      $1.000$ \\
    & 0.1 &   $0.929$ &      $0.979$ \\
    & / &   $0.357$ &      $0.708$ \\
SW & 0.01 &   $0.933$ &      $0.119$ \\
    & 0.05 &   $0.824$ &      $0.062$ \\
SF & 0.01 &   $0.941$ &      $0.135$ \\
    & 0.05 &   $0.813$ &      $0.073$ \\
LF & 0.01 &   $0.946$ &      $0.243$ \\
    & 0.05 &   $0.867$ &      $0.138$ \\
JB & 0.01 &   $1.000$ &      $0.256$ \\
    & 0.05 &   $1.000$ &      $0.181$ \\
DP & 0.01 &       NaN &      $0.230$ \\
    & 0.05 &       NaN &      $0.145$ \\
AD & 0.01 &   $1.000$ &      $0.133$ \\
    & 0.05 &   $0.988$ &      $0.072$ \\
CVM & 0.01 &   $0.000$ &      $0.164$ \\
    & 0.05 &   $0.000$ &      $0.093$ \\
FSSD & 0.01 &   $0.983$ &      $0.774$ \\
    & 0.05 &   $0.970$ &      $0.608$ \\
SBNN & / &   $0.285$ &      $0

### $\mathcal{R}_{earthquake}$

In [58]:
samples = datasets['R_earthquake']['samples']
labels = datasets['R_earthquake']['labels']
data = util.separate_by_size(samples)
ns = sorted(data.keys())

In [60]:
results = []
for n in ns:
    print(n)
    
    m = len(data[n])
    row = [n]

    for code in test_classifiers:
        print('{} '.format(code), end='')
        classifier = test_classifiers[code]
        s = sum([x == 0 for x in classifier.predict(data[n])])
        row.append(s / m)
    results.append(row)
    
    print('')

5
('SW', 0.01) ('SW', 0.05) ('SF', 0.01) ('SF', 0.05) ('LF', 0.01) ('LF', 0.05) ('JB', 0.01) ('JB', 0.05) ('DP', 0.01) skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 sa

Traceback (most recent call last):
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 200, in predict
    is_normal = self.test(sample, self.alpha)
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 163, in dp_test
    return scipy.stats.normaltest(sample)[1] >= alpha
  File "/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py", line 1702, in normaltest
    s, _ = skewtest(a, axis)
  File "/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py", line 1520, in skewtest
    raise ValueError(
ValueError: skewtest is not valid with less than 8 samples; 5 samples were given.
Traceback (most recent call last):
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 200, in predict
    is_normal = self.test(sample, self.alpha)
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 163, in dp_test
    return scipy.stats.normaltest

skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not vali

Traceback (most recent call last):
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 200, in predict
    is_normal = self.test(sample, self.alpha)
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 163, in dp_test
    return scipy.stats.normaltest(sample)[1] >= alpha
  File "/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py", line 1702, in normaltest
    s, _ = skewtest(a, axis)
  File "/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py", line 1520, in skewtest
    raise ValueError(
ValueError: skewtest is not valid with less than 8 samples; 5 samples were given.
Traceback (most recent call last):
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 200, in predict
    is_normal = self.test(sample, self.alpha)
  File "/home/milos/statisticka_klasifikacija/2020/github/util.py", line 163, in dp_test
    return scipy.stats.normaltest

skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not valid with less than 8 samples; 5 samples were given.
skewtest is not vali

...

('AD', 0.05) ('CVM', 0.01) ('CVM', 0.05) ('FSSD', 0.01) ('FSSD', 0.05) ('SBNN', '/') ('MMRT1', 0.01) ('MMRT1', 0.05) ('MMRT2', 0.01) ('MMRT2', 0.05) ('TTRT1', 0.01) ('TTRT1', 0.05) ('TTRT2', 0.01) ('TTRT2', 0.05) ('RSW', 0.01) ('RSW', 0.05) ('RLM', 0.01) ('RLM', 0.05) 
10
('SW', 0.01) ('SW', 0.05) ('SF', 0.01) ('SF', 0.05) ('LF', 0.01) ('LF', 0.05) ('JB', 0.01) ('JB', 0.05) ('DP', 0.01) 

/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


('DP', 0.05) ('AD', 0.01) ('AD', 0.05) ('CVM', 0.01) ('CVM', 0.05) ('FSSD', 0.01) ('FSSD', 0.05) ('SBNN', '/') ('MMRT1', 0.01) ('MMRT1', 0.05) ('MMRT2', 0.01) ('MMRT2', 0.05) ('TTRT1', 0.01) ('TTRT1', 0.05) ('TTRT2', 0.01) ('TTRT2', 0.05) ('RSW', 0.01) ('RSW', 0.05) ('RLM', 0.01) ('RLM', 0.05) 
15
('SW', 0.01) ('SW', 0.05) ('SF', 0.01) ('SF', 0.05) ('LF', 0.01) ('LF', 0.05) ('JB', 0.01) ('JB', 0.05) ('DP', 0.01) 

/home/milos/statisticka_klasifikacija/p39normal/lib/python3.9/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


('DP', 0.05) ('AD', 0.01) ('AD', 0.05) ('CVM', 0.01) ('CVM', 0.05) ('FSSD', 0.01) ('FSSD', 0.05) ('SBNN', '/') ('MMRT1', 0.01) ('MMRT1', 0.05) ('MMRT2', 0.01) ('MMRT2', 0.05) ('TTRT1', 0.01) ('TTRT1', 0.05) ('TTRT2', 0.01) ('TTRT2', 0.05) ('RSW', 0.01) ('RSW', 0.05) ('RLM', 0.01) ('RLM', 0.05) 
20
('SW', 0.01) ('SW', 0.05) ('SF', 0.01) ('SF', 0.05) ('LF', 0.01) ('LF', 0.05) ('JB', 0.01) ('JB', 0.05) ('DP', 0.01) ('DP', 0.05) ('AD', 0.01) ('AD', 0.05) ('CVM', 0.01) ('CVM', 0.05) ('FSSD', 0.01) ('FSSD', 0.05) ('SBNN', '/') ('MMRT1', 0.01) ('MMRT1', 0.05) ('MMRT2', 0.01) ('MMRT2', 0.05) ('TTRT1', 0.01) ('TTRT1', 0.05) ('TTRT2', 0.01) ('TTRT2', 0.05) ('RSW', 0.01) ('RSW', 0.05) ('RLM', 0.01) ('RLM', 0.05) 
25
('SW', 0.01) ('SW', 0.05) ('SF', 0.01) ('SF', 0.05) ('LF', 0.01) ('LF', 0.05) ('JB', 0.01) ('JB', 0.05) ('DP', 0.01) ('DP', 0.05) ('AD', 0.01) ('AD', 0.05) ('CVM', 0.01) ('CVM', 0.05) ('FSSD', 0.01) ('FSSD', 0.05) ('SBNN', '/') ('MMRT1', 0.01) ('MMRT1', 0.05) ('MMRT2', 0.01) ('MMRT2',

In [68]:
#codes = list(test_classifiers.keys())
results_df = pd.DataFrame(results, columns=['n'] + list(test_classifiers.keys()))
results_df = results_df.set_index('n')
results_df.columns = pd.MultiIndex.from_tuples(results_df.columns)

In [70]:
results_df

SW            SF            LF            JB            DP         \
      0.01   0.05   0.01   0.05   0.01   0.05   0.01   0.05   0.01   0.05   
n                                                                           
5    0.067  0.176  0.059  0.187  0.054  0.133  0.000  0.000  0.000  0.000   
10   0.201  0.421  0.186  0.389  0.112  0.283  0.078  0.144  0.212  0.305   
15   0.429  0.688  0.402  0.637  0.230  0.450  0.226  0.315  0.326  0.453   
20   0.602  0.840  0.539  0.792  0.326  0.564  0.317  0.440  0.400  0.566   
25   0.749  0.921  0.688  0.891  0.452  0.686  0.426  0.581  0.499  0.659   
30   0.872  0.962  0.812  0.947  0.534  0.765  0.525  0.671  0.580  0.738   
35   0.939  0.994  0.904  0.981  0.614  0.853  0.631  0.781  0.667  0.828   
40   0.972  0.997  0.946  0.989  0.715  0.905  0.699  0.837  0.719  0.871   
45   0.987  0.998  0.974  0.996  0.789  0.940  0.769  0.901  0.779  0.925   
50   0.995  1.000  0.986  1.000  0.862  0.974  0.824  0.937  0.828  0.950   
55   0.997  1.000  0.995  0.999  0.909  0.980  0.877  0.969  0.873  0.972   
60   1.000  1.000  1.000  1.000  0.940  0.994  0.928  0.991  0.929  0.994   
65   0.998  0.999  0.998  0.999  0.960  0.997  0.946  0.993  0.932  0.996   
70   1.000  1.000  0.999  1.000  0.967  0.995  0.962  0.996  0.954  0.997   
75   1.000  1.000  1.000  1.000  0.984  0.999  0.972  0.998  0.973  1.000   
80   1.000  1.000  1.000  1.000  0.991  0.999  0.986  0.999  0.983  0.998   
85   1.000  1.000  1.000  1.000  0.992  1.000  0.989  1.000  0.986  1.000   
90   1.000  1.000  1.000  1.000  0.999  1.000  0.992  1.000  0.993  1.000   
95   1.000  1.000  1.000  1.000  0.999  1.000  0.997  1.000  0.997  1.000   
100  1.000  1.000  1.000  1.000  1.000  1.000  0.999  1.000  0.999  1.000   

     ...  MMRT2         TTRT1         TTRT2           RSW           RLM         
     ...   0.01   0.05   0.01   0.05   0.01   0.05   0.01   0.05   0.01   0.05  
n    ...                                                                        
5    ...  0.021  0.100  0.056  0.159  0.055  0.162  0.052  0.141  0.002  0.008  
10   ...  0.138  0.367  0.145  0.250  0.167  0.321  0.116  0.225  0.001  0.005  
15   ...  0.242  0.582  0.215  0.360  0.324  0.545  0.194  0.339  0.001  0.001  
20   ...  0.260  0.697  0.170  0.329  0.324  0.569  0.228  0.378  0.027  0.035  
25   ...  0.371  0.772  0.231  0.375  0.435  0.712  0.276  0.442  0.013  0.017  
30   ...  0.420  0.854  0.293  0.432  0.612  0.813  0.328  0.480  0.001  0.002  
35   ...  0.625  0.905  0.329  0.449  0.703  0.888  0.398  0.542  0.000  0.000  
40   ...  0.657  0.944  0.320  0.522  0.724  0.889  0.423  0.566  0.013  0.015  
45   ...  0.722  0.955  0.331  0.485  0.812  0.934  0.441  0.612  0.004  0.007  
50   ...  0.831  0.983  0.372  0.504  0.866  0.967  0.495  0.650  0.001  0.004  
55   ...  0.880  0.983  0.381  0.519  0.918  0.977  0.514  0.701  0.002  0.004  
60   ...  0.904  0.996  0.399  0.614  0.930  0.973  0.562  0.716  0.001  0.003  
65   ...  0.927  0.994  0.415  0.580  0.934  0.986  0.581  0.732  0.001  0.004  
70   ...  0.956  0.994  0.396  0.540  0.968  0.988  0.630  0.761  0.000  0.001  
75   ...  0.954  0.998  0.408  0.548  0.980  0.997  0.654  0.775  0.000  0.000  
80   ...  0.973  0.999  0.463  0.691  0.982  0.994  0.689  0.822  0.001  0.002  
85   ...  0.987  0.998  0.490  0.671  0.990  0.998  0.679  0.813  0.001  0.001  
90   ...  0.989  1.000  0.485  0.612  0.995  1.000  0.714  0.829  0.002  0.002  
95   ...  0.999  1.000  0.419  0.567  0.995  1.000  0.763  0.874  0.000  0.000  
100  ...  0.998  1.000  0.589  0.918  0.996  1.000  0.754  0.860  0.001  0.001  

[20 rows x 29 columns]

In [84]:
df_res = pd.concat([dbnn_storage['results']['evaluation']['R_earthquake']['df'].T, 
                    results_df.T])

latex = util.get_latex_table(df_res.loc[:, list(range(10, 101, 10))],
                            index=True, 
                            caption='Rearthquake',
                            float_format='$%.2f$',
                            label='tab:R_earthquake')

print(latex)

\begin{table}
\centering
\caption{Rearthquake}
\label{tab:R_earthquake}
\begin{tabular}{llrrrrrrrrrr}
\toprule
    & n &    10  &    20  &    30  &    40  &    50  &    60  &    70  &    80  &    90  &    100 \\
\midrule
DBNN & 0.01 & $0.30$ & $0.73$ & $0.89$ & $0.95$ & $0.98$ & $0.99$ & $1.00$ & $0.99$ & $1.00$ & $1.00$ \\
    & 0.05 & $0.69$ & $0.94$ & $0.98$ & $0.99$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
    & 0.1 & $0.87$ & $0.98$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
    & / & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
    & opt & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
SW & 0.01 & $0.20$ & $0.60$ & $0.87$ & $0.97$ & $0.99$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
    & 0.05 & $0.42$ & $0.84$ & $0.96$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ & $1.00$ \\
SF & 0.01 & $0.19$ & $0.54$ & $0.81$ & $0.95$ & $0.99$ &